<a href="https://colab.research.google.com/github/miladziekanowska/Data_Analytics/blob/main/Pandas_Data_Cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cleaning data in Pandas DataFrame
Data cleaning is one of the tedious parts of work as a Data Analyst or Data Scientist, but it is one of the most important part of the the field - well prepared data can give us more insight and therefore allow us to make better informed decisions and train our models properly.

This notebook takes up quite a lot of examples of nessesary data cleaning with a few solutions, with the problems listed here:
1. Correcting the data type to our needs
2. Splitting the date and string columns
3. Handling missing data and NaN valeus
4. Detecting and dealing with outliers
5. Data Standarization
6. Data Discreditation
7. Data Normalization
8. Log Normalize
9. One-hot encoding
10. Data scaling
11. Profiling


Okay, let's get to work with the imports and first DataFrame.

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # for data visualization
import seaborn as sns # for statistical data visualization
import pylab 
import scipy.stats as stats
import datetime

We won't be doing any visualisations in this notebook I think, but it's a good practice to be prepared here. 

## 1. Correcting the columns
Before we jump in to any other cleaning and changes with our data, it is best to get to know it a little bit better. We need to see which columns have which type and decide, if we are going to group by some colums, or calculate some columns, etc. 

For this example, let's use the sales dataset from last notebook, this time I will translate all the name  of the colums do it's clearer.

In [26]:
sales_df = pd.read_csv('dane_sprzedaz.csv', sep=',', encoding='utf-8')
sales_df.columns = ['DATE', 'SHOP_ID', 'ITEM_ID', 'NET_PRICE', 'PROFIT_PERC', 'PROFIT']
sales_df

,DATE,SHOP_ID,ITEM_ID,NET_PRICE,PROFIT_PERC,PROFIT
0,01.05.2022,1,1,416.0,9,"37,44"
1,02.05.2022,1,1,454.0,2,"9,08"
2,03.05.2022,1,1,392.0,13,"50,96"
3,04.05.2022,1,1,498.0,9,"44,82"
4,05.05.2022,1,1,341.0,15,"51,15"
...,...,...,...,...,...,...
130,11.05.2022,3,3,374.0,7,"26,18"
131,12.05.2022,3,3,390.0,4,"15,6"
132,13.05.2022,3,3,485.0,10,"48,5"
133,14.05.2022,3,3,479.0,3,"14,37"


In [ ]:
# DataFrame ready, let's check the columns
sales_df.info()

We see that the DATE and PROFIT columns are in type object. While for date will split that in the next topic, or change into datetype data, first we need to take care of the profit column - this is how much earnings came from an item. Logically, this should be an float value, so we could get some statistics out of it.

In [ ]:
# let's first replace the coma with a dot, to have a proper form
sales_df['PROFIT'] = sales_df.PROFIT.str.replace(',', '.')
sales_df['PROFIT']

In [ ]:
# now that we have it in proper form, since this is still an object
sales_df['PROFIT'] = sales_df.PROFIT.astype(float)
sales_df['PROFIT']

In [ ]:
# another computation for that could be
sales_df['PROFIT'] = pd.to_numeric(sales_df['PROFIT'])
sales_df['PROFIT']

There are also two other function to change the data type in a column, for example if we would like to change the SHOP_ID value to string or object - since this is a shop's id, we won't be looking for maximum value or sum of the shops ids.   
Good explanation on two remaining methods can be found on [StackOverflow](https://stackoverflow.com/questions/15891038/change-column-type-in-pandas).  
There are also other Pandas methods like `.to_numeric()`, as presented below.

In [ ]:
# as mentioned, we could also change the DATE from an object type to a datetime type
sales_df['DATE'] = pd.to_datetime(sales_df['DATE'], errors = 'coerce', dayfirst=True)
# dayfirst=True is important if we want the dates in a European date structure - if we leave it out,
# it will provide us with the American, month-first structure
sales_df['DATE']

In [ ]:
# Let's go through one more example for quick corrections
df2 = pd.DataFrame({
    'NAME': ['APPLES', 'PEARS', 'STRAWBERRIES'],
    'PRICE': ['$ 3.22', '$ 3.22',  '$ 3.22']
})
df2.info()

The PRICE is once again, not a float but an object and it contains the $. We could replace it with, the same way we did last time `.replace('$ ', '')`, but let's go other route.

In [ ]:
remove_dol = lambda x: float(x[2:]) # we are trimming the values with this code
df2['PRICE_IN_DOLLARS'] = df2.PRICE.apply(remove_dol) # creating a new column with float values
df2

One more important and useful tip is to remember about the `.strip()` method for string values, which could help us avoid stupid duplicate values in object columns, especially if these were typed in by different people or from a survey and some might have pressed space on too many times.

## 2. Splitting data fom one to many columns and concatenating
Now that we have the sales_df columns in correct data types, let's see how we can get even more information from the date, as there are quite a few informations - year, month and day.

In [ ]:
# We need to create new columns for all the values we want - year, month and date
sales_df['YEAR'] = sales_df['DATE'].dt.year
sales_df['MONTH'] = sales_df['DATE'].dt.month
sales_df['DAY'] = sales_df['DATE'].dt.day
sales_df

Great! Now we can check what is the range from the first to last day, how many months and years are there, etc.


In [55]:
# Let's make another example, this time with string objects in the column
df = pd.DataFrame({
    'Address': ['4860 Sunset Boulevard,San Francisco,California',
                '3055 Paradise Lane,Salt Lake City,Utah',
                '682 Main Street,Detroit,Michigan',
                '9001 Cascade Road,Kansas City,Missouri']
})
df

,Address
0,"4860 Sunset Boulevard,San Francisco,California"
1,"3055 Paradise Lane,Salt Lake City,Utah"
2,"682 Main Street,Detroit,Michigan"
3,"9001 Cascade Road,Kansas City,Missouri"


In [60]:
# Now we see that all these Addresses have same template, so we want to split it
df['Address'].str.split(',', expand=True) # expand=True creates new columns from the split
# if we do not add it, we will get a list for reach row, where every split will create new element
# this is however only temporary view!

,0,1,2
0,4860 Sunset Boulevard,San Francisco,California
1,3055 Paradise Lane,Salt Lake City,Utah
2,682 Main Street,Detroit,Michigan
3,9001 Cascade Road,Kansas City,Missouri


In [70]:
# To create a safe columns, we would have to do is similar to the date, or we can use another computation
df = df.assign(Street = df['Address'].str.split(',', expand=True)[0],
               City = df['Address'].str.split(',', expand=True)[1],
               State = df['Address'].str.split(',', expand=True)[2])
# we can now delete the Address, since we will create a few new ;)
df = df.drop(['Address'], axis=1)

Now that we know how to split different data into new columns, what if we would like to join two or more columns?  
As with all concatenations, it is important that the concatenated data should be the same data type. If they aren't in the columns, we need to convert them in out concatenating code.

In [71]:
# Let's do one example from the DataFrame above
# To create a concatenated column, we of course need to create a new column
df['City_State'] = df['City'] + " - " + df['State']
df

,Street,City,State,City_State
0,4860 Sunset Boulevard,San Francisco,California,San Francisco - California
1,3055 Paradise Lane,Salt Lake City,Utah,Salt Lake City - Utah
2,682 Main Street,Detroit,Michigan,Detroit - Michigan
3,9001 Cascade Road,Kansas City,Missouri,Kansas City - Missouri


In [72]:
# the easiest concatenation ever
# let's try different syntax 
df['Address_New'] = df[['Street', 'City', 'State']].apply(', '.join, axis=1)
df

,Street,City,State,City_State,Address_New
0,4860 Sunset Boulevard,San Francisco,California,San Francisco - California,"4860 Sunset Boulevard, San Francisco, California"
1,3055 Paradise Lane,Salt Lake City,Utah,Salt Lake City - Utah,"3055 Paradise Lane, Salt Lake City, Utah"
2,682 Main Street,Detroit,Michigan,Detroit - Michigan,"682 Main Street, Detroit, Michigan"
3,9001 Cascade Road,Kansas City,Missouri,Kansas City - Missouri,"9001 Cascade Road, Kansas City, Missouri"


## 3. Handling missing values 
There are a few types of missing values in a DataFrame, which might cause some issues with the outcome and modeling.  
These values could be put in a following list:  
`falsy_values = (0, False, None, '', [], {})`  
Don't get it wrong, 0 within a numeric column often won't be a missing data - it is important to know the context of the column before we start to clean it. With numeric data, if 0 is and outlier, it still gives us some information and might not be a mistake - it could mean that, for example, a store was closed that day, therefore the total retails is 0. Even more so, if 0 is within the range, then we don't touch it, as this most likely a totally valid data, especially if there are not many of it. However, if we get a 0 within object columns, this might be a missing value or no answer.

Pandas also have their own version of missing values, which is *NaN*.   
The difference between NaN and None value is that if we have a NaN value in a numeric column, this won't print out errors, unlike None. However, NaN might affect the outcome of the operation.


In [ ]:
# Let's use a new DataFrame, that has missing values
dates = pd.date_range('20230401', periods=6)
df = pd.DataFrame(np.random.randn(6, 4), 
                  index=dates, 
                  columns=list('ABCD'))
df['D'] = -5
df['E'] = [np.nan, -1,-2,-3, -4, -5]
df['F'] = [1,2, np.nan, np.nan,np.nan, np.nan]
df

In [ ]:
# we can already see them, since this DataFrame is small, but let's check the info first
df.info()

Info method gives us quite a lot of useful information here.  
We see that in the column E there is one null-value and in column F there are only 2 non-null values - 2/3 of data is missing. While the one missing value in column E might not throw off our analysis (it might, if under this missing value hiddes an outlier), but with 2/3 of information missing are not good source of information either.

**What do we do, if we see many missing values?**  
1. We try to get these missing values from somewhere - from our client, from data engineers, from data bank, etc., wherever our data came from;
2. We try to solve these using bussiness logic - perhaps these missing values are for something - days off, system errors, someone forgot to input the data, maybe the item has been discontinued, etc.;
3. If we cannot get the data from anyone and we can't come up with a bussiness solution, we can fill these using statistics and math.

If we do not get the data from somewhere, based on the bussiness logic, we can do a few things with the missing data:
- *delete the rows with missing values* - it depends very much on the size of the dataset. Say we would have a few missing values in a large, or even medium-large data set, this shouldn't throw off the results too much. However, if that would throw off the distribution or does not cut out a whole category from another column. If these missing valeus are spead out pretty evenly and there are not many of them, we can delete them;
- *delete the whole column* - depending on situation and columns importance. If we are missing more than a half of the data in one column, we can't get them, and it's not crucial - perhaps this would be the best option. Or, we could delete the rows and work on a sample that contains the data;
- *ignore the missing values* - not the best solution overall, but also a possible approach. In this case, it would be best to provide the statistical values with and without the missing values in the report;
- *fill with "unknown"* - works better with categorical data than numerical, could be useful at the begining stage of the analysis, but it would differ everytime for the latter stages;
- *fill* - and we can distinguish quite a few ways to fill the unknown, besides "unknown":
  - fill with the value from above or from below;
  - fill with the mean or median;
  - fill with the result from interpolation;
  - fill with the data from a model.

While there are quite a lot of methods, everytime we get to know a new dataset and we find missing values, we need to carefully choose the most optimised approach to deal with it. There is no *one size fits all* solution.


In [76]:
# In here we have a practice dataset, so the actual outcome is not as important,
# therefore we will try a few solutions

In [ ]:
df[df.isna().any(axis=1)] # let's display all the rows with the missing values

In [ ]:
# now let's drop them
df.dropna() 

In [ ]:
# Not a lot is left in this dataset, right? The F column seems to be missing a lof of values. 
# Maybe it's better to just drop the F column
df.drop(['F'], axis=1) # much less NaNs

In [ ]:
# Okay, what if we don't want to drop the NaN values, but fill them with the upper one?
df.fillna(method='ffill')

In [ ]:
# There still is one NaN. The value is first in the column, so maybe lets fill it with the lower
df.fillna(method='bfill') # we didn't save the DataFrame, so we don't have the results from last action

In [ ]:
# Maybe let's not fill with the random data.
# How about we use the mean to use on the F column and median for E?
# Let's do it in a copy
df2 = df.copy()
df2['E'].fillna(df2['E'].median().round(1), inplace=True)
df2 # for E

In [ ]:
# and let's use different method to fill F with mean
df2['F'].replace([np.nan], df2['F'].mean(), inplace=True)
df2

In [ ]:
# At last, let's try interpolation
df.interpolate(method ='linear', limit_direction ='backward', inplace=True)
# similarely as with ffill and bfill, depending on the direction we might not catch 
# all the values at once
df.interpolate(method ='linear', limit_direction ='forward', inplace=True)
df

More on interpolation can be found [here](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.interpolate.html).

## 4. Detecting and dealing with outliers
